In [7]:
import xarray
import sys
sys.path.insert(0, '../src/')
import hdp
from os import listdir
import cftime
import numpy as np


EDF_OLD_OUTPUT = "/projects/dgs/persad_research/EDF_Methane_Scripts/heat/EDF_Methane_Mitigation_Heat/analysis_datasets/MMS_LENS2_TREFHTMN_TREFHTMX_HeatwaveMetrics.nc"
TREFHTMN_LENS2_BASELINE_DAILY_DIR = "/projects/dgs/persad_research/EDF_MMS_Data/LENS2/TREFHTMN/DAILY/1960_1999/"
TREFHTMN_LENS2_DAILY_DIR = "/projects/dgs/persad_research/EDF_MMS_Data/LENS2/TREFHTMN/DAILY/2015_2100/"


old_hwf = xarray.open_dataset(EDF_OLD_OUTPUT)["LENS2_TREFHTMN_HWF"]

In [11]:
start = cftime.DatetimeNoLeap(1961, 1, 1, 0, 0, 0, 0, has_year_zero=True)
end = cftime.DatetimeNoLeap(1990, 12, 31, 0, 0, 0, 0, has_year_zero=True)

baseline_temps = xarray.concat([xarray.open_mfdataset([TREFHTMN_LENS2_BASELINE_DAILY_DIR + name for name in listdir(TREFHTMN_LENS2_BASELINE_DAILY_DIR) if "." + str(index).zfill(3) in name and ".nc" in name]).sel(time=slice(start,end)) for index in range(1, 11)], dim="time")["TREFHTMN"]

new_times = baseline_temps.time.values
for index, time in enumerate(baseline_temps.time.values):
    new_times[index] = cftime.DatetimeNoLeap(int(index/365), time.month, time.day, 0, 0, 0, 0, has_year_zero=True)

baseline_temps = baseline_temps.assign_coords(time=new_times).load()

In [18]:
TREFHTMN_LENS2_DAILY_DIR = "/projects/dgs/persad_research/EDF_MMS_Data/LENS2/TREFHTMN/DAILY/2015_2100/"
threshold = hdp.compute_threshold(baseline_temps, np.array([0.95]))
datasets = []

for index in range(1, 11):
    print(index, end=", ")

    start = cftime.DatetimeNoLeap(2015, 1, 1, 0, 0, 0, 0, has_year_zero=True)
    end = cftime.DatetimeNoLeap(2099, 12, 31, 0, 0, 0, 0, has_year_zero=True)

    paths = [TREFHTMN_LENS2_DAILY_DIR + name for name in listdir(TREFHTMN_LENS2_DAILY_DIR) if f".{str(index).zfill(3)}." in name]
    lens2_trefhtmn = xarray.open_mfdataset(paths)["TREFHTMN"].sel(time=slice(start, end)).load()

    datasets.append(hdp.compute_heatwave_metrics(lens2_trefhtmn, threshold))

new_hw_ds = xarray.concat(datasets, dim="member")

1, 0.95, 2, 0.95, 3, 0.95, 4, 0.95, 5, 0.95, 6, 0.95, 7, 0.95, 8, 0.95, 9, 0.95, 10, 0.95, 

In [25]:
old_hwf

<xarray.DataArray 'LENS2_TREFHTMN_HWF' (member: 10, year: 85, lat: 192, lon: 288)>
[47001600 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * year     (year) int64 2015 2016 2017 2018 2019 ... 2095 2096 2097 2098 2099
  * member   (member) object '1181.010' '1001.001' ... '1141.008' '1161.009'
Attributes:
    temperature dataset path:      ['/projects/dgs/persad_research/EDF_MMS_Da...
    control dataset path:          /projects/dgs/persad_research/EDF_MMS_Data...
    time_created:                  2024-01-23 18:36:41.896221
    author:                        Cameron Cummins
    credit:                        Original algorithm written by Tammas Lough...
    Tammas Loughran's repository:  https://github.com/tammasloughran/ehfheatw...
    script repository:             https://github.com/AgentOxygen/heatwave_an...
    contact:                       cameron.cummins@utexas.edu
    description:                   Heatwave Frequency (HWF) for LENS2 TREFHTM...
    long_name:                     Heatwave Days per Year
    units:                         heatwave days

In [ ]:
import matplotlib.pyplot as plt

old_hwf = xarray.open_dataset(EDF_OLD_OUTPUT)["LENS2_TREFHTMN_HWD"]

f, ax1 = plt.subplots(1, 1, figsize=(10, 8), facecolor='w')

delta_ds = new_hw_ds["HWD"].weighted(np.cos(np.deg2rad(new_hw_ds.lat))).mean(dim=["member", "lat", "lon"]).sel(percentile=0.95)
delta_ds.plot(ax=ax1, color="Blue", label="New")
old_hwf.weighted(np.cos(np.deg2rad(old_hwf.lat))).mean(dim=["member", "lat", "lon"]).plot(ax=ax1, color="Red", label="Old")

ax1.legend()